LSTM

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


In [ ]:

# Baixar dados históricos da ação MGLU3 dos últimos 5 anos
mglu3 = yf.download('CMIN3.SA', period="5y")


In [ ]:

# Exibir os primeiros 5 registros
print(mglu3.head())

# Selecionar o preço ajustado
mglu3_close = mglu3['Adj Close'].dropna()


In [ ]:

# Criar um DataFrame com os valores ajustados
data = pd.DataFrame(mglu3_close)
data.columns = ['Close']


In [ ]:

# Normalizar os dados para os modelos
scaler = MinMaxScaler(feature_range=(0, 1))
data['Scaled_Close'] = scaler.fit_transform(data[['Close']])


In [ ]:

# Função para criar janelas de dados
def create_dataset(data, window_size=60):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)


In [ ]:

# Criar conjunto de dados
window_size = 60
scaled_data = data['Scaled_Close'].values
X, y = create_dataset(scaled_data, window_size)


In [ ]:

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:

# Ajustar o formato para o LSTM
X_train_lstm = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:

# Criar o modelo LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=1))


In [ ]:

# Compilar o modelo
lstm_model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:

# Treinar o modelo
lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=32, verbose=1)


In [ ]:

# Fazer previsões
lstm_predictions = lstm_model.predict(X_test_lstm)

# Inverter a normalização
lstm_predictions = scaler.inverse_transform(lstm_predictions.reshape(-1, 1))


Random Forest

In [ ]:

# Criar o modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:

# Fazer previsões
rf_predictions = rf_model.predict(X_test)

# Inverter a normalização
rf_predictions = scaler.inverse_transform(rf_predictions.reshape(-1, 1))


In [ ]:

# Valores reais
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:

# Calcular o erro médio quadrático (MSE)
lstm_mse = mean_squared_error(y_test_original, lstm_predictions)
rf_mse = mean_squared_error(y_test_original, rf_predictions)

print(f"LSTM MSE: {lstm_mse}")
print(f"Random Forest MSE: {rf_mse}")


Comparativo Entre os dois modelos

In [ ]:

# Obter as previsões para os próximos 5 dias
last_window = scaled_data[-window_size:]
last_window = last_window.reshape(1, window_size, 1)


In [ ]:

# Previsão do LSTM
lstm_forecasts = []
for _ in range(5):
    next_prediction = lstm_model.predict(last_window)
    lstm_forecasts.append(scaler.inverse_transform(next_prediction.reshape(-1, 1))[0][0])
    
    # Atualizar a janela de entrada para a próxima previsão
    last_window = np.append(last_window[:, 1:, :], next_prediction.reshape(1, 1, 1), axis=1)

    


In [ ]:

# Previsão do Random Forest
last_window_rf = scaled_data[-window_size:]
rf_forecasts = []
for _ in range(5):
    next_rf_prediction = rf_model.predict(last_window_rf.reshape(1, -1))
    rf_forecasts.append(scaler.inverse_transform(next_rf_prediction.reshape(-1, 1))[0][0])
    
    # Atualizar a janela de entrada para a próxima previsão
    last_window_rf = np.append(last_window_rf[1:], next_rf_prediction, axis=0)


In [ ]:

# Arredondar as previsões
lstm_forecasts = [round(pred, 2) for pred in lstm_forecasts]
rf_forecasts = [round(pred, 2) for pred in rf_forecasts]


In [ ]:

# Criar a tabela com pandas
days = [f"Dia {i + 1}" for i in range(5)]
forecast_data = {
    "Dia": days,
    "Previsão LSTM": lstm_forecasts,
    "Previsão Random Forest": rf_forecasts,
}

forecast_table = pd.DataFrame(forecast_data)


In [ ]:

# Criar o plot da tabela
plt.figure(figsize=(8, 4))
plt.axis('tight')
plt.axis('off')

# Adicionar a tabela ao plot
table_plot = plt.table(
    cellText=forecast_table.values,
    colLabels=forecast_table.columns,
    loc='center',
    cellLoc='center',
    colLoc='center'
)

# Estilizar a tabela
table_plot.auto_set_font_size(False)
table_plot.set_fontsize(10)
table_plot.auto_set_column_width(col=list(range(len(forecast_table.columns))))

# Exibir o plot
plt.title("Previsões para os Próximos 5 Dias", fontsize=14)
plt.show()


Duplicado?

In [ ]:

# print(f"LSTM MSE: {lstm_mse}")
# print(f"Random Forest MSE: {rf_mse}")

# # Obter as previsões para os próximos 5 dias
# last_window = scaled_data[-window_size:]
# last_window = last_window.reshape(1, window_size, 1)

# # Previsão do LSTM
# lstm_forecasts = []
# for _ in range(5):
#     next_prediction = lstm_model.predict(last_window)
#     lstm_forecasts.append(scaler.inverse_transform(next_prediction.reshape(-1, 1))[0][0])
    
#     # Atualizar a janela de entrada para a próxima previsão
#     last_window = np.append(last_window[:, 1:, :], next_prediction.reshape(1, 1, 1), axis=1)

# # Previsão do Random Forest
# last_window_rf = scaled_data[-window_size:]
# rf_forecasts = []
# for _ in range(5):
#     next_rf_prediction = rf_model.predict(last_window_rf.reshape(1, -1))
#     rf_forecasts.append(scaler.inverse_transform(next_rf_prediction.reshape(-1, 1))[0][0])
    
#     # Atualizar a janela de entrada para a próxima previsão
#     last_window_rf = np.append(last_window_rf[1:], next_rf_prediction, axis=0)

# # Arredondar as previsões
# lstm_forecasts = [round(pred, 2) for pred in lstm_forecasts]
# rf_forecasts = [round(pred, 2) for pred in rf_forecasts]

# # Criar a tabela com pandas
# days = [f"Dia {i + 1}" for i in range(5)]
# forecast_data = {
#     "Dia": days,
#     "Previsão LSTM": lstm_forecasts,
#     "Previsão Random Forest": rf_forecasts,
# }

# forecast_table = pd.DataFrame(forecast_data)

# # Criar o plot da tabela
# plt.figure(figsize=(8, 4))
# plt.axis('tight')
# plt.axis('off')

# # Adicionar a tabela ao plot
# table_plot = plt.table(
#     cellText=forecast_table.values,
#     colLabels=forecast_table.columns,
#     loc='center',
#     cellLoc='center',
#     colLoc='center'
# )

# # Estilizar a tabela
# table_plot.auto_set_font_size(False)
# table_plot.set_fontsize(10)
# table_plot.auto_set_column_width(col=list(range(len(forecast_table.columns))))

# # Exibir o plot
# plt.title("Previsões para os Próximos 5 Dias", fontsize=14)
# plt.show()
